# Efficient bigram search

Creates a table containing only bigrams, then demonstrates a search query on this table. Searching a single word on this table can be useful for a quick text check when building classifiers.

The table can be saved to a parquet file for easy loading.

In [1]:
import duckdb
from huggingface_hub import snapshot_download

from tqdm.auto import tqdm

/Users/kalyan/Library/Caches/pypoetry/virtualenvs/open-data-cnKQNmjn-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
REPO_NAME = "ClimatePolicyRadar/all-document-text-data"
REPO_URL = f"https://huggingface.co/datasets/{REPO_NAME}"
CACHE_DIR = "../cache"

REVISION = "main"  # Use this to set a commit hash. Recommended!

snapshot_download(
    repo_id=REPO_NAME,
    repo_type="dataset",
    local_dir=CACHE_DIR,
    revision=REVISION,
    allow_patterns=["*.parquet"],
)

Fetching 15 files: 100%|██████████| 15/15 [00:00<00:00, 2485.86it/s]


'/Users/kalyan/Documents/CPR/open-data/cache'

In [3]:
db = duckdb.connect()

# Authenticate (needed if loading a private dataset)
# You'll need to log in using `huggingface-cli login` in your terminal first
db.execute("CREATE SECRET hf_token (TYPE HUGGINGFACE, PROVIDER credential_chain);")

# Create a view called 'open_data', and count the number of rows and distinct documents
# in the view
db.execute(
    f"CREATE VIEW open_data AS SELECT * FROM read_parquet('{CACHE_DIR}/*.parquet')"
)
db.sql("SELECT COUNT(*), COUNT(DISTINCT document_id) FROM open_data")

┌──────────────┬─────────────────────────────┐
│ count_star() │ count(DISTINCT document_id) │
│    int64     │            int64            │
├──────────────┼─────────────────────────────┤
│     21037269 │                        7795 │
└──────────────┴─────────────────────────────┘

In [5]:
def create_bigram_table(conn, source_table: str, batch_size: int = 100000) -> None:
    # Get the total count of rows in the source table
    total_rows = conn.execute(f"SELECT COUNT(*) FROM {source_table}").fetchone()[0]

    conn.execute("""
        CREATE TABLE IF NOT EXISTS bigrams (
            bigram VARCHAR PRIMARY KEY,
            frequency INTEGER
        )
    """)

    # Process the data in batches
    for offset in tqdm(range(0, total_rows, batch_size), desc="Processing batches"):
        conn.execute(f"""
            INSERT INTO bigrams
            WITH words AS (
                SELECT unnest(string_split(lower("text_block.text"), ' ')) AS word
                FROM {source_table}
                LIMIT {batch_size} OFFSET {offset}
            ),
            bigram_pairs AS (
                SELECT 
                    word || ' ' || lead(word, 1) OVER (ORDER BY (SELECT NULL)) AS bigram
                FROM words
            ),
            batch_bigrams AS (
                SELECT bigram, COUNT(*) AS frequency
                FROM bigram_pairs
                WHERE bigram IS NOT NULL
                GROUP BY bigram
            )
            SELECT bigram, frequency FROM batch_bigrams
            ON CONFLICT (bigram) DO UPDATE SET
            frequency = bigrams.frequency + EXCLUDED.frequency
        """)

    # Create an index on the bigram column for faster searching (if not already created by PRIMARY KEY)
    conn.execute("CREATE INDEX IF NOT EXISTS bigram_idx ON bigrams(bigram)")


create_bigram_table(db, "open_data")

Processing batches:   0%|          | 0/211 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 211/211 [04:06<00:00,  1.17s/it]


In [7]:
stopwords = [
    "i",
    "me",
    "my",
    "myself",
    "we",
    "our",
    "ours",
    "ourselves",
    "you",
    "your",
    "yours",
    "yourself",
    "yourselves",
    "he",
    "him",
    "his",
    "himself",
    "she",
    "her",
    "hers",
    "herself",
    "it",
    "its",
    "itself",
    "they",
    "them",
    "their",
    "theirs",
    "themselves",
    "what",
    "which",
    "who",
    "whom",
    "this",
    "that",
    "these",
    "those",
    "am",
    "is",
    "are",
    "was",
    "were",
    "be",
    "been",
    "being",
    "have",
    "has",
    "had",
    "having",
    "do",
    "does",
    "did",
    "doing",
    "a",
    "an",
    "the",
    "and",
    "but",
    "if",
    "or",
    "because",
    "as",
    "until",
    "while",
    "of",
    "at",
    "by",
    "for",
    "with",
    "about",
    "against",
    "between",
    "into",
    "through",
    "during",
    "before",
    "after",
    "above",
    "below",
    "to",
    "from",
    "up",
    "down",
    "in",
    "out",
    "on",
    "off",
    "over",
    "under",
    "again",
    "further",
    "then",
    "once",
    "here",
    "there",
    "when",
    "where",
    "why",
    "how",
    "all",
    "any",
    "both",
    "each",
    "few",
    "more",
    "most",
    "other",
    "some",
    "such",
    "no",
    "nor",
    "not",
    "only",
    "own",
    "same",
    "so",
    "than",
    "too",
    "very",
    "s",
    "t",
    "can",
    "will",
    "just",
    "don",
    "should",
    "now",
]

stopwords_condition = " AND ".join(
    [f"bigram NOT SIMILAR TO '.*\\b{word}\\b.*'" for word in stopwords]
)


def search_bigrams(db, search_term: str, limit: int = 100) -> list[tuple[str, int]]:
    result = db.execute(f"""
        SELECT DISTINCT bigram, frequency
        FROM bigrams
        WHERE bigram LIKE '%{search_term.lower()}%'
        AND {stopwords_condition}
        ORDER BY frequency DESC, bigram
        LIMIT {limit} 
    """)

    return result.fetchall()


search_bigrams(db, "climate", 10)

[('climate change', 16323),
 ('climate change.', 2365),
 ('climate change,', 2146),
 ('national climate', 1595),
 ('climate policy', 932),
 ('climate risks', 734),
 ('climate finance', 662),
 ('climate adaptation', 659),
 ('climate action', 647),
 ('climate financing', 528)]

In [8]:
db.sql("COPY bigrams TO 'bigrams.parquet' (FORMAT PARQUET);")